### This notebook was intended to be ran on Google Colab

If it isn't, then remove the drive mount, and update the paths

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
#EleutherAI/pythia-70m
#EleutherAI/pythia-1b
#EleutherAI/pythia-1.4b

#load dataset

dataset_path = r""#dataset path
dataset_name = ""#jsonl name
model_name = ""#either one of the pretrained ^ or a checkpointfile
pre_model_name = "" #hugging face name ^
output_dir = "" #output folder

In [ ]:
!pip install datasets
!pip install PyYAML
!pip install wheel
!pip install pytorch
!pip install transformers
!pip install pandas
!pip install transformers

import os
import datasets
from datasets import load_dataset
import tempfile
import logging
import random
import yaml
import time
import torch
import transformers
import pandas as pd
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import Trainer
from transformers import TrainingArguments
from transformers import Trainer

logger = logging.getLogger(__name__)
global_config = None

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  error: subprocess-exi

In [ ]:
from huggingface_hub import login
login(token="Sorry, cant show you this :(")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
training_config = {
    "model": {
        "pretrained_name": pre_model_name,
        "max_length" : 1024
    },
    "datasets": {
        "use_hf": False,
        "path": dataset_path
    },
    "verbose": True
}

In [ ]:


dataset = load_dataset("json", data_files=dataset_path, split='train')
tokenizer = AutoTokenizer.from_pretrained(pre_model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def encode(examples):
    tokenized = tokenizer(examples["question"], truncation=True, padding="max_length",max_length=700)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized
tokenized_dataset = dataset.map(encode, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/1759 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name,attn_implementation='eager')
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=2, #epochs here
    per_device_train_batch_size=10,
    per_device_eval_batch_size=40,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    remove_unused_columns=False
)
model_flops = (
  model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

In [ ]:
trainer = trainer.train()

Step,Training Loss


In [ ]:
import time
for i in range(10000):
  time.sleep(10)
  print("kernel stay alive please")

In [ ]:
path = r"" #find checkpoint in output folder
model = AutoModelForCausalLM.from_pretrained(path)
model_name = "" #model name
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
input_prompt = r"" #your input
input_ids = tokenizer.encode(input_prompt, return_tensors='pt')
attention_mask = torch.ones_like(input_ids)

# Generate 5 different sequences using beam search
output_sequence = model.generate(
    input_ids,
    num_beams=1,
    num_return_sequences=1,
    temperature=10,
    attention_mask = attention_mask,
    pad_token_id = tokenizer.pad_token_id,
    max_length=1000)


response = tokenizer.decode(output_sequence[0])

In [ ]:
response